In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [2]:
!pip install -q keras

In [3]:
import keras

Using TensorFlow backend.


In [4]:
from tensorflow.keras.preprocessing.image import img_to_array, load_img, ImageDataGenerator
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten, Dropout
from tensorflow.keras.models import Model
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
%matplotlib inline
import argparse
import os
from imutils import paths

IMAGE_SIZE = [224,224]

test_path = '/content/drive/My Drive/Colab Notebooks/cell/Test'
train_path = '/content/drive/My Drive/Colab Notebooks/cell/Train'


In [5]:
import tensorflow

In [6]:
#setting the images into same size 
train_datagen = ImageDataGenerator(rescale = 1./255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)

In [7]:
test_datagen = ImageDataGenerator(rescale = 1./255)

In [8]:
training_set = train_datagen.flow_from_directory('/content/drive/My Drive/Colab Notebooks/cell/Train',target_size=(244,244),batch_size=32,class_mode='categorical')

Found 3833 images belonging to 2 classes.


In [9]:
test_set = test_datagen.flow_from_directory('/content/drive/My Drive/Colab Notebooks/cell/Test',target_size=(244,244),batch_size=32,class_mode='categorical')

Found 977 images belonging to 2 classes.


In [10]:
trainImages = list(paths.list_images('/content/drive/My Drive/Colab Notebooks/cell/Train'))

In [11]:
testImages = list(paths.list_images('/content/drive/My Drive/Colab Notebooks/cell/Test'))

In [12]:
#importing images into file
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
train = []
labels = []
for i in trainImages: 
  label = i.split(os.path.sep)[-2]
  img = load_img(i, target_size = (244, 244), interpolation = "nearest", color_mode = "rgb")
  img = img_to_array(img)
  img = preprocess_input(img)
  train.append(img)
  labels.append(label)


/usr/local/lib/python3.6/dist-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


In [13]:
train = np.array(train)
labels = np.array(labels)

In [14]:
from sklearn.preprocessing import LabelBinarizer
from keras.utils.np_utils import to_categorical   
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
labels = to_categorical(labels)

In [15]:
from sklearn.model_selection import train_test_split
X_Train, X_Test, y_train, y_test = train_test_split(train, labels, train_size = 0.75, random_state = 40, stratify = labels)

In [16]:
print(X_Train.shape, X_Test.shape, y_train.shape, y_test.shape)

(2874, 244, 244, 3) (959, 244, 244, 3) (2874, 2) (959, 2)


In [17]:
IDG = ImageDataGenerator(rotation_range = 50, width_shift_range = 0.5, height_shift_range = 0.5, horizontal_flip = True, vertical_flip = True, fill_mode = 'nearest')

In [18]:
from tensorflow.keras.applications import MobileNetV2
base = MobileNetV2(alpha = 1.0, include_top = False, weights = 'imagenet',input_tensor = Input(shape = (224, 224, 3)), classes = 2)

9412608/9406464 [==============================] - 0s 0us/step


In [19]:
finalModel = base.output
from tensorflow.keras.layers import AveragePooling2D, Dropout, Flatten, Dense
finalModel = AveragePooling2D(pool_size=(7, 7))(finalModel)
finalModel = Flatten(name = "flatten")(finalModel)
finalModel = Dense(128, activation = "relu")(finalModel)
finalModel = Dropout(0.5)(finalModel)
finalModel = Dense(2, activation = "softmax")(finalModel)

In [20]:
model = Model(inputs = base.input, outputs = finalModel)

In [21]:
for layers in base.layers:
  layers.trainable = False

In [22]:
INIT_LR = 1e-4
EPOCHS = 40
BS = 32
from tensorflow.keras.optimizers import Adamax
model.compile(loss = "categorical_crossentropy", optimizer = Adamax(lr = INIT_LR, decay = INIT_LR // EPOCHS), metrics = ["accuracy"])


In [23]:
main = model.fit(IDG.flow(X_Train, y_train, batch_size = BS), validation_data = (X_Test, y_test), validation_steps = len(X_Test) // BS, epochs = EPOCHS, steps_per_epoch = len(X_Train) // BS)

Epoch 1/40
89/89 [==============================] - 177s 2s/step - loss: 0.6737 - accuracy: 0.6358 - val_loss: 0.4710 - val_accuracy: 0.8269
Epoch 2/40
89/89 [==============================] - 176s 2s/step - loss: 0.4818 - accuracy: 0.7685 - val_loss: 0.3313 - val_accuracy: 0.9176
Epoch 3/40
89/89 [==============================] - 176s 2s/step - loss: 0.4115 - accuracy: 0.8100 - val_loss: 0.2646 - val_accuracy: 0.9364
Epoch 4/40
89/89 [==============================] - 177s 2s/step - loss: 0.3387 - accuracy: 0.8522 - val_loss: 0.2161 - val_accuracy: 0.9437
Epoch 5/40
89/89 [==============================] - 175s 2s/step - loss: 0.3078 - accuracy: 0.8765 - val_loss: 0.1829 - val_accuracy: 0.9531
Epoch 6/40
89/89 [==============================] - 175s 2s/step - loss: 0.2921 - accuracy: 0.8790 - val_loss: 0.1606 - val_accuracy: 0.9593
Epoch 7/40
89/89 [==============================] - 177s 2s/step - loss: 0.2859 - accuracy: 0.8878 - val_loss: 0.1470 - val_accuracy: 0.9625
Epoch 8/40
89

In [24]:
y_pred = model.predict(x = X_Test, batch_size = BS)

In [25]:
y_pred = np.argmax(y_pred, axis=1)

In [26]:
y_test = y_test.argmax(axis = 1)

In [27]:
from sklearn.metrics import r2_score
r_squared = r2_score(y_test, y_pred)

In [28]:
print(r_squared)

0.8957245998608212


In [29]:
model.save('finalversion.h5')